<a href="https://colab.research.google.com/github/Apyarn95/Kinship-Detection/blob/main/DeepFaceKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! gdown https://drive.google.com/uc?id=1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo
from keras.models import Model
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout
from PIL import Image
import numpy as np
from keras_vggface.vggface import VGGFace


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Downloading...
From: https://drive.google.com/uc?id=1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo
To: /content/vgg_face_weights.h5
580MB [00:13, 41.8MB/s]


In [ ]:
#required libraries
!pip install keras_vggface
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
from random import choice, sample
import cv2
from imageio import imread
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model
from keras import regularizers
from keras.layers import Input, Embedding, LSTM, Dropout, BatchNormalization,Dense, concatenate, Flatten, Conv1D
from keras.optimizers import RMSprop, Adam
from keras_vggface.vggface import VGGFace
from glob import glob
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.preprocessing import image
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D, Lambda, Reshape
from collections import defaultdict
from keras_vggface.utils import preprocess_input
    

In [ ]:
TRAIN_BASE = '/content/drive/My Drive/train'    #folder containing training data
families = sorted(os.listdir(TRAIN_BASE))
print('We have {} families in the dataset'.format(len(families)))

#all_images contains paths of images.
all_images = glob(TRAIN_BASE + "*/*/*/*.jpg")

#Splitting the data into train & validation set.
#Validation set includes family with name F09.
val_families = "F09"
train_images = [x for x in all_images if val_families not in x]
val_images = [x for x in all_images if val_families in x]

We have 470 families in the dataset


In [ ]:
ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]


In [ ]:
#val stands for validation process
train_person_to_images_map = defaultdict(list)

for x in train_images:
  train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

val_person_to_images_map = defaultdict(list)

for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)

relationships = pd.read_csv('/content/drive/My Drive/train_relationships.csv')
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if val_families not in x[0]]
val = [x for x in relationships if val_families in x[0]]  

In [ ]:
# Facenet architecture will take image of size 160 x 160
IMG_SIZE_FN = 160

#Facenet architecture wil take image of size 224 x 224
IMG_SIZE_VGG = 224

input_1 = Input(shape=(IMG_SIZE_FN,IMG_SIZE_FN,3))  #facenet for image1
input_2 = Input(shape=(IMG_SIZE_FN,IMG_SIZE_FN,3))  #facenet for image 2
input_3 = Input(shape=(IMG_SIZE_VGG,IMG_SIZE_VGG,3))  #VGG for image 1
input_4 = Input(shape=(IMG_SIZE_VGG,IMG_SIZE_VGG,3))  #VGG for image 2


In [ ]:
#!pip install git+https://github.com/rcmalli/keras-vggface.git
from keras.models import load_model
# load the model
facenet_model = load_model('/content/drive/My Drive/facenet_keras.h5')
fn_1 = facenet_model(input_1)
fn_2 = facenet_model(input_2)
#vgg_model = VGGFace(model='resnet50', include_top=False)
vgg_1 = vgg_model(input_3)
vgg_2 = vgg_model(input_4)
    
x1 = Reshape((1, 1 ,128))(fn_1)   #reshaping image array for global max pool layer
x2 = Reshape((1, 1 ,128))(fn_2)
x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

#the below 4 lamda functions will calcluate the square of each input image
lambda_1 = Lambda(lambda tensor  : K.square(tensor))(fn_1) 
lambda_2 = Lambda(lambda tensor  : K.square(tensor))(fn_2)
lambda_3 = Lambda(lambda tensor  : K.square(tensor))(vgg_1)
lambda_4 = Lambda(lambda tensor  : K.square(tensor))(vgg_2)
    
added_facenet = Add()([x1, x2])     #this function will add two images image 1 image 2 given by facenet architecture
added_vgg = Add()([vgg_1, vgg_2])    #this function will add two images image 3 image 4 given by VGG architecture
subtract_fn = Subtract()([x1,x2])    #this function will subtract two images image 1 image 2 given by facenet architecture
subtract_vgg = Subtract()([vgg_1,vgg_2])   #this function will subtract two images image 3 image 4 given by VGG architecture
subtract_fn2 = Subtract()([x2,x1])    #this function will subtract two images image 2 image 1 given by facenet architecture
subtract_vgg2 = Subtract()([vgg_2,vgg_1])   #this function will subtract two images image 4 image 3 given by VGG architecture
prduct_fn1 = Multiply()([x1,x2])    #this function will multiply two images image 1 image 2 given by facenet architecture
prduct_vgg1 = Multiply()([vgg_1,vgg_2])   #this function will multiply two images image 3 image 4 given by VGG architecture
sqrt_fn1 = Add()([lambda_1,lambda_2])        # this function implements x1^2 + x2^2 where x1 and x2 are image by facenet
sqrt_vgg1 = Add()([lambda_3,lambda_4])       # this function implements vgg_1^2 + vgg_2^2 where vgg_1 and vgg_2 are image by VGG
sqrt_fn2 = Lambda(lambda tensor  : K.sign(tensor)*K.sqrt(K.abs(tensor)+1e-9))(prduct_fn1) #squre_root of sqrt_fn1
sqrt_vgg2 = Lambda(lambda tensor  : K.sign(tensor)*K.sqrt(K.abs(tensor)+1e-9))(prduct_vgg1) #squre_root of sqrt_vgg1

    
added_vgg = Conv2D(128 , [1,1] )(added_vgg)
subtract_vgg = Conv2D(128 , [1,1] )(subtract_vgg)
subtract_vgg2 = Conv2D(128 , [1,1] )(subtract_vgg2)
prduct_vgg1 = Conv2D(128 , [1,1] )(prduct_vgg1)
sqrt_vgg1 = Conv2D(128 , [1,1] )(sqrt_vgg1)
sqrt_vgg2 = Conv2D(128 , [1,1] )(sqrt_vgg2)
    
#finally concatenating all the above featues for final layer which is to be inputed to the dense layers.
concatenated= Concatenate(axis=-1)([Flatten()(added_vgg), (added_facenet), Flatten()(subtract_vgg), (subtract_fn),
                                   Flatten()(subtract_vgg2), (subtract_fn2), Flatten()(prduct_vgg1), (prduct_fn1), 
                                   Flatten()(sqrt_vgg1), (sqrt_fn1), Flatten()(sqrt_vgg2), (sqrt_fn2)])
    
concatenated= Dense(500, activation="relu")(concatenated)
concatenated= Dropout(0.1)(concatenated)
concatenated= Dense(100, activation="relu")(concatenated)
concatenated= Dropout(0.1)(concatenated)
concatenated= Dense(25, activation="relu")(concatenated)
concatenated= Dropout(0.1)(concatenated)
out = Dense(1, activation="sigmoid")(concatenated) #output sigmoid layer

#defining the model
model = Model([input_1, input_2, input_3, input_4], out)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [ ]:

# ‘read_img_fn’ will take the path of the input image and return the same image with a 
# predefined size. Remember, we are using different input sizes of images for different
#  base models. Like that, another function ‘read_img_vgg’ will do the same for the VGGFace 
#  model.

def read_img_fn(path):
    img = cv2.imread(path)
    img = cv2.resize(img,(IMG_SIZE_FN,IMG_SIZE_FN))
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

#this function will read image from specified path and convert it into required size
def read_img_vgg(path):
    img = cv2.imread(path)
    img = cv2.resize(img,(IMG_SIZE_VGG,IMG_SIZE_VGG))
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

In [ ]:
def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1_FN = np.array([read_img_fn(x) for x in X1])
        X1_VGG = np.array([read_img_vgg(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2_FN = np.array([read_img_fn(x) for x in X2])
        X2_VGG = np.array([read_img_vgg(x) for x in X2])

        yield [X1_FN, X2_FN, X1_VGG, X2_VGG], labels  

In [ ]:
file_path = '/content/drive/My Drive/checkpoint'
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)

callbacks_list = [checkpoint, reduce_on_plateau]

#resuming training Load checkpoint:
import os.path
if os.path.exists(file_path):
    # Load model:
    model.load_weights(file_path)

model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00003))



In [ ]:

model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=10, verbose=1,
                    workers = 4,callbacks=callbacks_list, steps_per_epoch=200,validation_steps=100)

/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the `keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/10
200/200 [==============================] - 756s 4s/step - loss: 1.1224 - acc: 0.5409 - val_loss: 0.6434 - val_acc: 0.5869

Epoch 00001: val_acc improved from -inf to 0.58688, saving model to /content/drive/My Drive/checkpoint
Epoch 2/10
200/200 [==============================] - 317s 2s/step - loss: 0.6588 - acc: 0.6269 - val_loss: 0.6198 - val_acc: 0.6338

Epoch 00002: val_acc improved from 0.58688 to 0.63375, saving model to /content/drive/My Drive/checkpoint
Epoch 3/10
200/200 [==============================] - 228s 1s/step - loss: 0.6216 - acc: 0.6566 - val_loss: 0.6749 - val_acc: 0.6800

Epoch 00003: val_acc improved from 0.63375 to 0.68000, saving model to /content/drive/My Drive/checkpoint
Epoch 4/10
200/200 [==============================] - 215s 1s/step - loss: 0.5940 - acc: 0.6834 - val_loss: 0.6910 - val_acc: 0.7025

Epoch 00004: val_acc improved from 0.68000 to 0.70250, saving model to /content/drive/My Drive/checkpoint
Epoch 5/10
200/200 [=======================

In [ ]:
Ksi = cv2.imread("/content/drive/My Drive/Ff.jpg")

Ksi_FN = cv2.resize(Ksi,(IMG_SIZE_FN,IMG_SIZE_FN))
Ksi_FN = np.array(Ksi_FN).astype(np.float)
Ksi_Vgg = cv2.resize(Ksi,(IMG_SIZE_VGG,IMG_SIZE_VGG))
Ksi_Vgg = np.array(Ksi_Vgg).astype(np.float)
X1_FN = np.array(preprocess_input(Ksi_FN,version=2))
X1_Vgg = np.array(preprocess_input(Ksi_Vgg,version=2))
# X1 = [x.split("-")[0] for x in batch]
# X1_FN = np.array([read_img_fn(test_path + x) for x in X1])
# X1_VGG = np.array([read_img_vgg(test_path + x) for x in X1])
X1_FN = np.expand_dims(X1_FN,axis=0)
X1_Vgg = np.expand_dims(X1_Vgg,axis=0)


In [ ]:
Fa = cv2.imread("/content/drive/My Drive/Logan.jpg")
Fa_FN = cv2.resize(Fa,(IMG_SIZE_FN,IMG_SIZE_FN))
Fa_FN = np.array(Fa_FN).astype(np.float)
Fa_Vgg = cv2.resize(Fa,(IMG_SIZE_VGG,IMG_SIZE_VGG))
Fa_Vgg = np.array(Fa_Vgg).astype(np.float)
X2_FN = np.array(preprocess_input(Fa_FN,version=2))
X2_Vgg = np.array(preprocess_input(Fa_Vgg,version=2))
X2_FN = np.expand_dims(X2_FN,axis=0)
X2_Vgg = np.expand_dims(X2_Vgg,axis=0)


In [ ]:
pred = model.predict([X1_FN,X2_FN,X1_Vgg,X2_Vgg])
# model.summary()


In [ ]:
Ksi.show()

AttributeError: ignored

array([[0.45489734]], dtype=float32)